In [ ]:
%load_ext aiida
%aiida
import urllib.parse as urlparse
import matplotlib.pyplot as plt

import ipywidgets as ipw
from aiida import orm
from aiidalab_widgets_base import viewer

from surfaces_tools import helpers
from surfaces_tools.widgets import comments, obsolete

In [ ]:
pk = urlparse.parse_qs(urlparse.urlsplit(jupyter_notebook_url).query)["pk"][0]
workcalc = orm.load_node(pk)
summary= workcalc.outputs.report.get_dict()['summary']
timings= workcalc.outputs.report.get_dict()['min_times_per_nnodes']

# Summary

In [ ]:
print(summary)

# Plot

In [ ]:
# Create plots with pre-defined labels.
nodes=[]
times=[]
speedup=[]
speedup_ideal=[]
speedup_50=[]
for nnodes, time in timings.items():
    nodes.append(int(nnodes))
    times.append(time)
speedup = [times[0]/i for i in times]
speedup_ideal = [i/nodes[0] for i in nodes]
speedup_50 = [i/(2*nodes[0]) for i in nodes]
fig, ax = plt.subplots()
ax.plot(nodes, speedup, 'ro-', label='benchmark speedup')
ax.plot(nodes, speedup_ideal, 'go-', label='ideal speedup')
ax.plot(nodes, speedup_50, 'yo-', label='50% ideal speedup')
ax.set_xlabel('# nodes')
ax.set_ylabel('speedup')
legend = ax.legend(loc='upper center', shadow=True, fontsize='x-large')

# Put a nicer background color on the legend.
legend.get_frame().set_facecolor('C7')

plt.show()
#plt.savefig('speedup.png',dpi=300)

# Comments

In [ ]:
comments_widget = comments.CommentsWidget(workchain=pk)
display(comments_widget)

# Mark calculation as obsolete 

In [ ]:
obsolete = obsolete.ObsoleteWidget(workchain=pk)
display(obsolete)